In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Simulated financial data for 6 companies
data = {
    'Company': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Revenue': [1000, 1200, 900, 1100, 1150, 980],
    'Expenses': [600, 750, 500, 700, 730, 580],
    'Profit': [400, 450, 400, 400, 420, 400],
    'Stock_Price': [150, 170, 145, 160, 165, 155],
    'Profit_Margin': [0.4, 0.375, 0.444, 0.364, 0.365, 0.408]
}
df = pd.DataFrame(data)


In [ ]:
print("Missing Values:\n", df.isnull().sum())


In [ ]:
# Simulate missing value
df.loc[2, 'Expenses'] = np.nan

# Fill missing value with mean
df['Expenses'].fillna(df['Expenses'].mean(), inplace=True)


In [ ]:
correlation_matrix = df.drop(columns=['Company']).corr()


In [ ]:
print("\nCorrelation Matrix:\n", correlation_matrix)


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True)
plt.title('Financial Metrics Correlation Heatmap')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap with Coolwarm Palette')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='viridis',
    linewidths=1,
    linecolor='white'
)
plt.title('Financial Correlation Heatmap with Borders')
plt.show()


In [ ]:
# If needed, mask missing values:
mask = correlation_matrix.isnull()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', mask=mask)
plt.title('Correlation Heatmap with Missing Values Masked')
plt.show()


In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define a custom diverging colormap
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", ["#d73027", "#ffffbf", "#1a9850"])

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap=custom_cmap, linewidths=0.5)
plt.title('Custom Colormap: Financial Metrics Correlation')
plt.show()
